<a href="https://colab.research.google.com/github/choobrendan/Flag-Guesser/blob/main/flag_guess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from keras import regularizers
from keras.layers import LeakyReLU
from PIL import Image
keras = tf.keras

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp  /content/Flags.zip
!unzip PetImages.zip

cp: missing destination file operand after '/content/Flags.zip'
Try 'cp --help' for more information.
unzip:  cannot find or open PetImages.zip, PetImages.zip.zip or PetImages.zip.ZIP.


In [4]:
gpus = tf.config.experimental.list_physical_devices('CPU')


In [5]:
gpus

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [6]:
gpus = tf.config.experimental.list_physical_devices ('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth (gpu, True)

In [7]:
import cv2
import imghdr
from matplotlib import pyplot as plt

In [8]:
data_dir=r"/content/drive/MyDrive/Flags"

In [9]:
from struct import unpack
from tqdm import tqdm
import os


marker_mapping = {
    0xffd8: "Start of Image",
    0xffe0: "Application Default Header",
    0xffdb: "Quantization Table",
    0xffc0: "Start of Frame",
    0xffc4: "Define Huffman Table",
    0xffda: "Start of Scan",
    0xffd9: "End of Image"
}


class JPEG:
    def __init__(self, image_file):  # Fix the method name here
        with open(image_file, 'rb') as f:
            self.img_data = f.read()

    def decode(self):
        data = self.img_data
        while(True):
            marker, = unpack(">H", data[0:2])
            # print(marker_mapping.get(marker))
            if marker == 0xffd8:
                data = data[2:]
            elif marker == 0xffd9:
                return
            elif marker == 0xffda:
                data = data[-2:]
            else:
                lenchunk, = unpack(">H", data[2:4])
                data = data[2+lenchunk:]
            if len(data)==0:
                break


bads = []

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        image = JPEG(image_path)
        try:
          image.decode()
        except:
          bads.append(image_path)
          print(image_path)


for name in bads:
  try:
    os.remove(image_path)
  except Exception:
      pass

In [10]:
image_exts=["jpeg","jpg","bmp","png"]

In [11]:
num_classes=len(os.listdir(r"/content/drive/MyDrive/Flags"))

In [12]:

def convert_to_png(image_path):
    img = Image.open(image_path)
    new_image_path = os.path.splitext(image_path)[0] + ".png"
    img.save(new_image_path, "PNG")
    os.remove(image_path)  # Remove the original non-image file

for image_class in os.listdir(data_dir):
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try:
            tip = imghdr.what(image_path)
            if tip not in image_exts:
                print('Image not in ext list {}'.format(image_path))
                convert_to_png(image_path)
        except (IOError, OSError) as e:
            print('Issue with image {}'.format(image_path))

Image not in ext list /content/drive/MyDrive/Flags/Belize/vector5.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector5.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize/vector.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize/vector6.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector6.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize/vector7.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector7.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize/vector3.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector3.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize/vector2.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector2.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize/vector4.svg
Issue with image /content/drive/MyDrive/Flags/Belize/vector4.svg
Image not in ext list /content/drive/MyDrive/Flags/Belize

In [13]:
datagen = ImageDataGenerator(
    rescale=1./255,
    zoom_range=0.2,         # Randomly zoom images by 20%
    width_shift_range=0.1,  # Randomly shift images horizontally by 10%
    height_shift_range=0.1, # Randomly shift images vertically by 10%
    rotation_range=5,      # Randomly rotate images by up to 20 degrees
    brightness_range=(0.7, 1.3),  # Randomly adjust brightness between 0.8 and 1.2
    shear_range=0.2,        # Randomly apply shearing transformations
    horizontal_flip=False,   # Randomly flip images horizontally
    vertical_flip=False      # Randomly flip images vertically
)

In [14]:
data = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    label_mode='categorical',  # This indicates multiclass classification
    seed=42,  # Set the seed for reproducibility
    image_size=(256, 256),
    batch_size=32
)

Found 13863 files belonging to 80 classes.


In [15]:
data=data.map(lambda x,y:(x/255,y))

In [16]:
scaled_iterator=data.as_numpy_iterator()

In [17]:
batch=scaled_iterator.next()

In [18]:
train_size = int(len(data) * 0.7)
val_size = int(len(data) * 0.15)
test_size = int(len(data) * 0.15)

In [19]:
train=data.take (train_size)
val = data.skip(train_size).take(val_size)
test=data.skip (train_size+val_size).take(test_size)

In [20]:
# def preprocess_image(image_path):
#     img = cv2.imread(image_path)
#     tip = imghdr.what(image_path)
#     if tip not in image_exts:
#         print('Image not in ext list {}'.format(image_path))
#         os.remove(image_path)
#         return None
#     # Convert to 8-bit unsigned integers
#     img = cv2.convertScaleAbs(img)
#     return img

In [21]:
import tensorflow as tf

# Define the data augmentation function
def augment_image(image, label):
    # Randomly adjust brightness, contrast, saturation, and hue
    image = tf.image.random_brightness(image, max_delta=0.02)
    image = tf.image.random_contrast(image, lower=0.99, upper=1.01)
    image = tf.image.random_saturation(image, lower=0.99, upper=1.01)
    image = tf.image.random_hue(image, max_delta=0.01)
    # Add more augmentation techniques as per your requirement
    # For example, you can use tf.image.random_flip_left_right for random horizontal flipping, etc.

    return image, label

# Define the preprocess function without resizing to use it later for validation and test sets
def preprocess_image(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.uint8)  # Convert to 8-bit unsigned int
    image = tf.image.resize(image, (256, 256))
    image =image/255.0# Resize to model input size
    return image, label

# # Apply duplication and augmentation to the training data
# def augment_and_duplicate(image, label, num_duplicates=1):
#     # List to store augmented images and labels
#     augmented_images = []
#     augmented_labels = []

#     # Add original image and label to the lists
#     augmented_images.append(image)
#     augmented_labels.append(label)

#     # Apply augmentation and add duplicates to the lists
#     for _ in range(num_duplicates):
#         augmented_image, augmented_label = augment_image(image, label)
#         augmented_images.append(augmented_image)
#         augmented_labels.append(augmented_label)

#     # Stack the lists to create tensors
#     augmented_images = tf.stack(augmented_images)
#     augmented_labels = tf.stack(augmented_labels)

#     return augmented_images, augmented_labels

# # Duplicate and augment the training data
# num_duplicates = 2 # You can adjust the number of duplicates as per your requirement
# train = train.flat_map(lambda image, label: tf.data.Dataset.from_tensor_slices(augment_and_duplicate(image, label, num_duplicates)))
train = train.map(preprocess_image)
# Apply preprocessing to validation and test sets
val = val.map(preprocess_image)
test = test.map(preprocess_image)


In [22]:
model = Sequential()
model.add(Conv2D(32, (9, 9), 1, activation=keras.layers.LeakyReLU(alpha=0.2), input_shape=(256, 256, 3)))
model.add(MaxPooling2D())


# Second convolution layer with increased filters, Dropout, and Regularization
model.add(Conv2D(64, (7, 7), strides=1, activation=keras.layers.LeakyReLU(alpha=0.15), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.25))
model.add(MaxPooling2D())

# Third convolution layer with increased filters and BatchNormalization
model.add(Conv2D(64, (5, 5), strides=1, activation=keras.layers.LeakyReLU(alpha=0.10), kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# Third convolution layer with increased filters and BatchNormalization
model.add(Conv2D(32, (3, 3), strides=1, activation=keras.layers.LeakyReLU(alpha=0.05), kernel_regularizer=regularizers.l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D())

# Flatten the output to feed into the dense layers
model.add(Flatten())

# Dense layers with Dropout and Regularization
model.add(Dense(256, activation=keras.layers.LeakyReLU(alpha=0.02), kernel_regularizer=regularizers.l2(0.01)))
model.add(Dropout(0.5))

# Output layer for multiclass classification
model.add(Dense(num_classes, activation='softmax'))

early_stopping=tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0,
    patience=30,
    verbose=1,
    mode='min',
    baseline=None,
    restore_best_weights=True,
    start_from_epoch=0
)

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],)



In [23]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 248, 248, 32)      7808      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 124, 124, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 118, 118, 64)      100416    
                                                                 
 dropout (Dropout)           (None, 118, 118, 64)      0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 59, 59, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 55, 55, 64)        1

In [24]:
logdir='logs'

In [ ]:
tensorboard_callback=tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist=model.fit(train, epochs=500, validation_data=val, callbacks =[tensorboard_callback,early_stopping])

Epoch 1/500
303/303 [==============================] - 152s 466ms/step - loss: 6.7728 - accuracy: 0.5361 - val_loss: 3.9572 - val_accuracy: 0.7654
Epoch 2/500
303/303 [==============================] - 154s 505ms/step - loss: 3.1633 - accuracy: 0.7369 - val_loss: 2.2643 - val_accuracy: 0.8139
Epoch 3/500
303/303 [==============================] - 139s 452ms/step - loss: 2.2185 - accuracy: 0.7922 - val_loss: 1.8310 - val_accuracy: 0.8630
Epoch 4/500
303/303 [==============================] - 141s 461ms/step - loss: 1.9680 - accuracy: 0.8098 - val_loss: 1.8778 - val_accuracy: 0.8356
Epoch 5/500
303/303 [==============================] - 154s 506ms/step - loss: 1.9011 - accuracy: 0.8241 - val_loss: 2.3914 - val_accuracy: 0.6817
Epoch 6/500
303/303 [==============================] - 155s 508ms/step - loss: 1.7877 - accuracy: 0.8430 - val_loss: 1.7274 - val_accuracy: 0.8668
Epoch 7/500
303/303 [==============================] - ETA: 0s - loss: 1.7526 - accuracy: 0.8564

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
# Assuming you have already trained the model and have the test dataset
# (or you can use 'test' set if you have already created it)

# Get the true labels and predicted labels from the test set
y_true = []
y_pred = []
for images, labels in test:
    y_true.extend(np.argmax(labels, axis=1))  # Get the true class indices
    y_pred.extend(np.argmax(model.predict(images), axis=1))  # Get the predicted class indices

# Compute the confusion matrix
conf_matrix = confusion_matrix(y_true, y_pred)

# Normalize the confusion matrix to show probabilities instead of counts
conf_matrix_norm = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis]
class_names = os.listdir(r"/content/drive/MyDrive/Flags")
# Plot the heatmap
plt.figure(figsize=(40, 32))
sns.heatmap(conf_matrix_norm, annot=True, cmap="Blues", fmt=".2f", xticklabels=class_names, yticklabels=class_names)
plt.title('Confusion Matrix - Probabilities')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()
# If you want a classification report (precision, recall, F1-score, etc.), you can use the following:

report = classification_report(y_true, y_pred, target_names=class_names)

print("Classification Report:")
print(report)

In [ ]:
new_image_path = r"/content/sample_data/TestImage.jpg"  # Replace with the path to your new image

i=23
# Load and preprocess the new image
new_image = cv2.imread(new_image_path)
new_image = cv2.convertScaleAbs(new_image)
new_image = cv2.cvtColor(new_image, cv2.COLOR_RGB2BGR)
new_image = tf.image.convert_image_dtype(new_image, dtype=tf.uint8)
resize= tf.image.resize(new_image,(256,256))
# new_image = new_image / 255.0  # Normalize pixel values to [0, 1]
predictions=model.predict(np.expand_dims(resize,axis=0))
# Convert the image to a supported depth (e.g., 8-bit unsigned integers)

#  # Add batch dimension

# View the new image
plt.imshow(new_image.numpy().squeeze())
plt.axis('off')
plt.show()

predicted_class_index = np.argmax(predictions[0])
predicted_class_name = predicted_class_name = sorted(os.listdir(r"/content/drive/MyDrive/Flags"))[predicted_class_index] # Assuming class indices are integers starting from 0

# Print the predicted class index and name
print("Predicted Class Index:", predicted_class_index)
print("Predicted Class Name:", predicted_class_name)
print("Predicted Class Confidence:", predictions[0][np.argmax(predictions[0])])
predictions


In [ ]:
a=os.listdir(r"/kaggle/input/flag-pictures-dataset/Flags")
sorted(a)

In [ ]:
predictions

In [ ]:


batch[2][31]

In [ ]:
new_image

In [ ]:


# Convert the image to a supported depth (e.g., 8-bit unsigned integers)
image = cv2.convertScaleAbs((new_image/(255)).numpy(), alpha=(255.0))
image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
image = np.expand_dims(image, axis=0)  # Add batch dimension

# View the new image
plt.imshow(cv2.cvtColor(image.squeeze(), cv2.COLOR_BGR2RGB))
plt.axis('off')
plt.show()


In [ ]:
model.save(r"/content/drive/MyDrive")

In [ ]:
modelsaved = tf.keras.models.load_model(r"/content/sequential_3")

In [ ]:
%cd /content/sequential_3

In [ ]:
!zip -r sequential_3.zip /content/sequential_3/


In [ ]:
from google.colab import files
files.download(r'/content/sequential_3/sequential_3.zip')